In [1]:
import numpy as np
import pandas as pd
train_data = np.load('../input/bert-embeddings-train.npy')
train_label = pd.read_csv('../../Disaster/train.csv')['target']
test_data = np.load('../input/bert-embeddings-test.npy')

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_data,train_label)

In [ ]:
test_labels = logreg.predict(test_data)

In [ ]:
len(test_labels)

In [ ]:
import pandas as pd
sample_file = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample_file['target'] = test_labels

In [ ]:
sample_file

In [ ]:
sample_file.to_csv('result.csv', index=False)

In [ ]:
pd.read_csv('result.csv')

In [3]:
import lightgbm as lgb
d_train = lgb.Dataset(train_data, label=train_label)

params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

In [4]:
y_pred=clf.predict(test_data)

In [11]:
import pandas as pd
sample_file = pd.read_csv('../input/sample_submission.csv')
sample_file['target'] = [int(round(x)) for x in y_pred]
sample_file.to_csv('result_lgbm.csv', index=False)

In [24]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt

# sklearn tools for model training and assesment
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score)

import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'metric_freq': 1,
    'is_training_metric': True,
    'max_bin': 255,
    'learning_rate': 0.1,
    'num_leaves': 120,
    'tree_learner': 'serial',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 5,
    'is_enable_sparse': True,
    'use_two_round_loading': False,
    'is_save_binary_file': False,
    'output_model': 'LightGBM_model.txt',
    'num_machines': 1,
    'local_listen_port': 12400,
    'machine_list_file': 'mlist.txt',
    'verbose': 0,
    'subsample_for_bin': 200000,
    'min_child_samples': 20,
    'min_child_weight': 0.001,
    'min_split_gain': 0.0,
    'colsample_bytree': 1.0,
    'reg_alpha': 0.0,
    'reg_lambda': 0.0
}


lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# train
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval)

gridParams = {
    'learning_rate': [ 0.1],
    'num_leaves': [80],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary']
}

mdl = lgb.LGBMClassifier(
    task = params['task'],
    metric = params['metric'],
    metric_freq = params['metric_freq'],
    is_training_metric = params['is_training_metric'],
    max_bin = params['max_bin'],
    tree_learner = params['tree_learner'],
    feature_fraction = params['feature_fraction'],
    bagging_fraction = params['bagging_fraction'],
    bagging_freq = params['bagging_freq'],
    min_data_in_leaf = params['min_data_in_leaf'],
    min_sum_hessian_in_leaf = params['min_sum_hessian_in_leaf'],
    is_enable_sparse = params['is_enable_sparse'],
    use_two_round_loading = params['use_two_round_loading'],
    is_save_binary_file = params['is_save_binary_file'],
    n_jobs = -1
)

scoring = {'AUC': 'roc_auc'}

# Create the grid
grid = GridSearchCV(mdl, gridParams, verbose=2, cv=5, scoring=scoring, n_jobs=-1, refit='AUC')
# Run the grid
grid.fit(train_data, train_label)

print('Best parameters found by grid search are:', grid.best_params_)
print('Best score found by grid search is:', grid.best_score_)

(6090, 768) (6090,)
(1523, 768) (1523,)
[1]	valid_0's auc: 0.791822
[2]	valid_0's auc: 0.80984
[3]	valid_0's auc: 0.821787
[4]	valid_0's auc: 0.821499
[5]	valid_0's auc: 0.824656
[6]	valid_0's auc: 0.828532
[7]	valid_0's auc: 0.834399
[8]	valid_0's auc: 0.837061
[9]	valid_0's auc: 0.839102
[10]	valid_0's auc: 0.842026
[11]	valid_0's auc: 0.843226
[12]	valid_0's auc: 0.844236
[13]	valid_0's auc: 0.84352
[14]	valid_0's auc: 0.844198
[15]	valid_0's auc: 0.846793
[16]	valid_0's auc: 0.849239
[17]	valid_0's auc: 0.851167
[18]	valid_0's auc: 0.85151
[19]	valid_0's auc: 0.851635
[20]	valid_0's auc: 0.853208
[21]	valid_0's auc: 0.851871
[22]	valid_0's auc: 0.852675
[23]	valid_0's auc: 0.854051
[24]	valid_0's auc: 0.852964
[25]	valid_0's auc: 0.853117
[26]	valid_0's auc: 0.854322
[27]	valid_0's auc: 0.853442
[28]	valid_0's auc: 0.853288
[29]	valid_0's auc: 0.853249
[30]	valid_0's auc: 0.853469
[31]	valid_0's auc: 0.853757
[32]	valid_0's auc: 0.853898
[33]	valid_0's auc: 0.853727
[34]	valid_0's 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.8min finished


Best parameters found by grid search are: {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'num_leaves': 80, 'objective': 'binary'}
Best score found by grid search is: 0.8599550284072338


In [52]:
from keras.models import Sequential


from keras.layers import Dense, BatchNormalization, Dropout
model = Sequential([
    Dense(256, activation='relu', input_shape=(768,)),
    BatchNormalization(),
    Dropout(0.2),
  Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
  Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),
  Dense(1, activation='sigmoid'),
])
model.compile(
  optimizer='nadam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6090, 768) (6090,)
(1523, 768) (1523,)


In [57]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
6090/6090 [==============================] - 2s 381us/step - loss: 0.0916 - accuracy: 0.9655
Epoch 2/100
6090/6090 [==============================] - 2s 360us/step - loss: 0.0946 - accuracy: 0.9634
Epoch 3/100
6090/6090 [==============================] - 2s 394us/step - loss: 0.0961 - accuracy: 0.9629
Epoch 4/100
6090/6090 [==============================] - 2s 394us/step - loss: 0.0911 - accuracy: 0.9655
Epoch 5/100
6090/6090 [==============================] - 2s 373us/step - loss: 0.0928 - accuracy: 0.9650
Epoch 6/100
6090/6090 [==============================] - 3s 464us/step - loss: 0.0945 - accuracy: 0.9622
Epoch 7/100
6090/6090 [==============================] - 3s 451us/step - loss: 0.0965 - accuracy: 0.9635
Epoch 8/100
6090/6090 [==============================] - 2s 381us/step - loss: 0.0897 - accuracy: 0.9667
Epoch 9/100
6090/6090 [==============================] - 2s 362us/step - loss: 0.0866 - accuracy: 0.9675
Epoch 10/100
6090/6090 [==============================]

6090/6090 [==============================] - 1s 240us/step - loss: 0.0724 - accuracy: 0.9724
Epoch 79/100
6090/6090 [==============================] - 1s 236us/step - loss: 0.0633 - accuracy: 0.9750
Epoch 80/100
6090/6090 [==============================] - 1s 237us/step - loss: 0.0688 - accuracy: 0.9734
Epoch 81/100
6090/6090 [==============================] - 1s 239us/step - loss: 0.0686 - accuracy: 0.9736
Epoch 82/100
6090/6090 [==============================] - 1s 240us/step - loss: 0.0639 - accuracy: 0.9759
Epoch 83/100
6090/6090 [==============================] - 1s 239us/step - loss: 0.0717 - accuracy: 0.9736
Epoch 84/100
6090/6090 [==============================] - 1s 241us/step - loss: 0.0569 - accuracy: 0.9785
Epoch 85/100
6090/6090 [==============================] - 1s 238us/step - loss: 0.0691 - accuracy: 0.9732
Epoch 86/100
6090/6090 [==============================] - 1s 237us/step - loss: 0.0691 - accuracy: 0.9729
Epoch 87/100
6090/6090 [==============================] - 2

In [58]:
g = model.predict(X_test)
g = np.transpose(g)[0]
prediction = [int(round(x)) for x in g]

In [59]:
import keras
from sklearn.metrics import classification_report
keras.metrics.accuracy(y_test, prediction)
model.evaluate(X_test, y_test, verbose=0)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82       860
           1       0.77      0.72      0.75       663

    accuracy                           0.79      1523
   macro avg       0.79      0.78      0.78      1523
weighted avg       0.79      0.79      0.79      1523



In [61]:
y_pred=model.predict(test_data)
import pandas as pd
sample_file = pd.read_csv('../input/sample_submission.csv')
y_pred = np.transpose(y_pred)[0]
sample_file['target'] = [int(round(x)) for x in y_pred]
sample_file.to_csv('result_lgbm.csv', index=False)